<td>
   <a target="_blank" href="https://www.clarifai.com/" ><img src="https://upload.wikimedia.org/wikipedia/commons/b/bc/Clarifai_Logo_FC_Web.png" width=256/></a>
</td>

<td>
<a href="https://colab.research.google.com/github/Clarifai/examples/blob/main/models/model_predict.ipynb" target="_blank"><img
src="https://colab.research.google.com/assets/colab-badge.svg" alt="Colab"></a>
</td>


This notebook provides an introduction to using the Clarifai SDK to interact with prediction APIs for various AI models. 

To get predictions for an input, you need to supply an input and the model you'd like to get predictions from. 
Input can be formed from any of url or filepath or raw bytes

Notebook is structured into sections based on the type of input the models accept
 - Text
 - Image
 - Video
 - Audio

The final section delves into the potential of configuring additional parameters (such as threshold and concepts) to enhance the flexibility of prediction operations.

# Getting Started

### Installation

In [ ]:
! pip install clarifai

In [1]:
import os
os.environ["CLARIFAI_PAT"] = "PAT" # replace with your own PAT key here

*Note: Guide to get your [PAT](https://docs.clarifai.com/clarifai-basics/authentication/personal-access-tokens)*

### For Colab
To access data files from Clarifai examples repo, you can clone the repo

In [ ]:
!git clone https://github.com/Clarifai/examples.git
%cd /content/examples/

### Listing of Models


Here's an example on how to list all models in Clarifai community filtered by model_type, description

In [ ]:
from clarifai.client.app import App

all_llm_community_models = App().list_models(filter_by={"query": "LLM",
                                                        "model_type_id": "text-to-text"}, only_in_app=False)

for llm_model in all_llm_community_models:
  print("Model ID ", llm_model.id)

### Get Model Details

Below cell is how to know more on details of model(description, usecases..etc) and info on training or other inference parameters(eg: temperature, top_k, max_tokens..etc for LLMs)

In [ ]:
from clarifai.client.user import User
gpt_4_model = User(user_id="openai").app(app_id="chat-completion").model(model_id="GPT-4")
print(gpt_4_model)

# Model Inference

Model class objects can be inititalised by providing its URL or also by defining respective `user_id`, `app_id` and `model_id`

In [2]:
# import Model class
from clarifai.client.model import Model

In [ ]:
model = Model("https://clarifai.com/clarifai/main/models/general-image-recognition")
                    # or
model = Model(user_id="clarifai", app_id="main", model_id="general-image-recognition")

## Text

### Text-to-Text

Model takes text data as input by filename or bytes or url.

Below is an example of how you would send text bytes/filepath and receive predictions from Clarifai's hosted models

#### Predict by Bytes

**Text Generation** - Generating text from an LLM

In [ ]:
# Asking Claude2 to write a tweet on future of AI
model_url = "https://clarifai.com/anthropic/completion/models/claude-v2"
model_prediction = Model(model_url).predict_by_bytes(b"Write a tweet on future of AI", input_type="text")

# Get the output
print(model_prediction.outputs[-1].data.text.raw)

#### Predict by FilePath

**Text Classification** - Sentiment Classification for given text input

In [ ]:
# Text sentiment analysis with 3 classes positive, negative, neutral.
model_url = "https://clarifai.com/erfan/text-classification/models/sentiment-analysis-twitter-roberta-base"
file_path = "datasets/upload/data/text_files/positive/0_9.txt"
model_prediction = Model(model_url).predict_by_filepath(file_path, input_type="text")

# Get the output
print(model_prediction.outputs[-1].data.concepts)

### Text-To-Image

**Image Generation** - Generate an Image from given input Prompt

In [ ]:
# Image Generation using Stable Diffusion XL
model_url = "https://clarifai.com/stability-ai/stable-diffusion-2/models/stable-diffusion-xl"
model_prediction = Model(model_url).predict_by_bytes(b"A painting of a cat", input_type="text")

In [12]:
# Display the image
import numpy as np
import cv2
import matplotlib.pyplot as plt

# Base64 image to numpy array
im_b = model_prediction.outputs[0].data.image.base64
image_np = np.frombuffer(im_b, np.uint8)
img_np = cv2.imdecode(image_np, cv2.IMREAD_COLOR)  

plt.axis('off')
plt.imshow(img_np[...,::-1])

### Text-To-Audio
Generate Audio from given text

In [ ]:
model_url = "https://clarifai.com/eleven-labs/audio-generation/models/speech-synthesis"

model_prediction = Model(model_url).predict_by_bytes(b"Hello, How are you doing today!", "text")

# Save the audio file
with open('output_audio.wav', mode='bx') as f:
    f.write(model_prediction.outputs[0].data.audio.base64)

## Image

### Image-To-Text

**Image Captioning** - Generate Caption for the given input Image

In [ ]:
model_url = "https://clarifai.com/salesforce/blip/models/general-english-image-caption-blip"
image_url = "https://s3.amazonaws.com/samples.clarifai.com/featured-models/image-captioning-statue-of-liberty.jpeg"
model_prediction = Model(model_url).predict_by_url(image_url, input_type="image")

# Get the output
print(model_prediction.outputs[0].data.text.raw)

**Image classification** - Below is an example of how you would send image URL and receive predictions from Clarifai's general-image-recognition model.

In [ ]:
model_url = "https://clarifai.com/clarifai/main/models/general-image-recognition"
image_url = "https://samples.clarifai.com/metro-north.jpg"
model_prediction = Model(model_url).predict_by_url(image_url, input_type="image")

# Get the output
print(model_prediction.outputs[0].data)

## Video

Here's an example of inference on video input where we also define `sample_ms` parameter to run prediction every 2000ms

In [ ]:
BEER_VIDEO_URL = "https://samples.clarifai.com/beer.mp4"

model = Model("https://clarifai.com/clarifai/main/models/general-image-recognition",
      output_config={
          "sample_ms": 2000 #Run inference every 2 seconds
      })

model_prediction = model.predict_by_url(BEER_VIDEO_URL, input_type="video")

# Print the frame info and the first concept name in each frame
for frame in model_prediction.outputs[0].data.frames:
    print(f"Frame Info: {frame.frame_info} Concept: {frame.data.concepts[0].name}\n")


## Audio

### Audio to Text

Transcribe Audio to text

In [ ]:
audio_url = "https://s3.amazonaws.com/samples.clarifai.com/GoodMorning.wav"
model_url = "https://clarifai.com/facebook/asr/models/asr-wav2vec2-large-robust-ft-swbd-300h-english"
model_prediction = Model(model_url).predict_by_url(audio_url, "audio")

# Print the output
print(model_prediction.outputs[0].data.text.raw)

## Prediction Parameters
You can set additional parameters to gain flexibility in the predict operation.


### Select Concepts
By putting this additional parameter on your predict calls, you can receive predict value(s) for only the concepts that you want to. 

You can specify particular concepts by either their id and/or their name

In [29]:
from clarifai_grpc.grpc.api import resources_pb2

In [32]:
DOG_IMAGE_URL = "https://samples.clarifai.com/dog2.jpeg"
Model_URL = "https://clarifai.com/clarifai/main/models/general-image-recognition"

In [30]:
selected_concepts = [
      resources_pb2.Concept(name="dog"),
      resources_pb2.Concept(name="cat"),
  ]
model = Model(Model_URL,
    output_config={
        "select_concepts": selected_concepts
    })

model_prediction = model.predict_by_url(DOG_IMAGE_URL, input_type="image")

# Print the output
print(model_prediction.outputs[0].data.concepts)

### Maximum Concepts

Setting the `max_concepts` parameter will customize how many concepts and their corresponding probability scores the predict endpoint will return. 

If not specified, the predict endpoint will return the top 20 concepts

In [33]:
model = Model(Model_URL,
      output_config={
          "max_concepts": 3
      })

model_prediction = model.predict_by_url(DOG_IMAGE_URL, input_type="image")

# Print the output
print(model_prediction.outputs[0].data.concepts)

### Minimum Prediction Value

This parameter lets you set a minimum probability threshold for the outputs you want to view for the Predict operation.

For example if you want to see all concepts with a probability score of .95 or higher, this parameter will allow you to accomplish that.

In [ ]:
model = Model(Model_URL,
        output_config={
            "min_value": 0.95
        })
model_prediction = model.predict_by_url(DOG_IMAGE_URL, input_type="image")

# Print the output
print(model_prediction.outputs[0].data.concepts)

### By Model Version ID
By specifying model_version_id in your predict call, you can continue to predict on a previous version, for consistent prediction results. Clarifai also updates its pre-built models on a regular basis.

Below is an example of how you would set a model version ID and receive predictions from Clarifai's `general-image-recognition` model.

In [ ]:
model_version_id = "aa7f35c01e0642fda5cf400f543e7c40"
model_url = f"https://clarifai.com/clarifai/main/models/general-image-recognition/model_version/{model_version_id}"

model = Model(model_url)

model_prediction = model.predict_by_url(DOG_IMAGE_URL, input_type="image")

# Print the output
print(model_prediction.outputs[0].data.concepts)

## Clarifai Resources

**Website**: [https://www.clarifai.com](https://www.clarifai.com/)

**Demo**: [https://clarifai.com/demo](https://clarifai.com/demo)

**Sign up for a free Account**: [https://clarifai.com/signup](https://clarifai.com/signup)

**Developer Guide**: [https://docs.clarifai.com](https://docs.clarifai.com/)

**Clarifai Community**: [https://clarifai.com/explore](https://clarifai.com/explore)

**Python SDK Docs**: [https://clarifai-python.readthedocs.io/en/latest/index.html](https://clarifai-python.readthedocs.io/en/latest/index.html)

---